In [ ]:
#Installer tous les packages nécaissaires
!pip install -r ../requirements.txt

#ou bien

# !pip install autocorrect
# !pip install -U spacy
# !python -m spacy download fr_core_news_sm
# !pip install pyspellchecker
# !pip install keras==2.11.0
# !pip install xgboost
# !pip install h2o

In [3]:
import openpyxl
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import nltk
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer  
from nltk.corpus import stopwords
import spacy
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

In [4]:
file4 = '../resources/common/data/labled_comments.xlsx'
df_labled = pd.read_excel(file4)
file5 = 'resources/common/CleanedData/Lematized_Comments.xlsx'
df = pd.read_excel (file5)

In [5]:
df.columns

Index(['comment', 'tokens', 'clean_tokens', 'clean_data_noTokenized ',
       'clean_data_noTokenized', 'lemmatized_tokens'],
      dtype='object')

In [6]:
df_labled.columns

Index(['comment', 'score'], dtype='object')

# Semi-supervised Learning with LSTM

In [4]:
# Étape 1: Entraînez un modèle sur les données étiquetées

small_texts = df_labled['comment'].values
small_labels = df_labled['score'].values

# Encodage des étiquettes
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(small_labels)
small_y = to_categorical(encoded_labels)

# Tokenization et padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(small_texts)
sequences = tokenizer.texts_to_sequences(small_texts)
word_index = tokenizer.word_index
small_X = pad_sequences(sequences)

# Construction du modèle
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=small_X.shape[1]))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(small_y.shape[1], activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînement du modèle
model.fit(small_X, small_y, epochs=10, batch_size=32)

# Étape 2: Utilisez le modèle pour prédire des étiquettes pour les données non étiquetées

# Supposons que df soit votre DataFrame contenant les données non étiquetées
# et que la colonne 'lemmatized_tokens' contient les textes déjà nettoyés, tokenisés et lemmatisés

# Recombiner les tokens en texte
unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens'].values]
unlabeled_sequences = tokenizer.texts_to_sequences(unlabeled_texts)
unlabeled_X = pad_sequences(unlabeled_sequences, maxlen=small_X.shape[1])

# Prédiction
predictions = model.predict(unlabeled_X)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels)

# Étape 3: Combinez les données étiquetées originales avec les données auto-étiquetées
combined_texts = np.concatenate((small_texts, unlabeled_texts))
combined_labels = np.concatenate((small_labels, predicted_labels))

# Encodage des étiquettes combinées
encoded_combined_labels = label_encoder.transform(combined_labels)
combined_y = to_categorical(encoded_combined_labels)

# Tokenization et padding des textes combinés
combined_sequences = tokenizer.texts_to_sequences(combined_texts)
combined_X = pad_sequences(combined_sequences, maxlen=small_X.shape[1])

# Étape 4: Ré-entraînez le modèle sur le nouvel ensemble de données combiné
model.fit(combined_X, combined_y, epochs=10, batch_size=32)
# Préparer les données de la grande dataset pour la prédiction
# Comme précédemment mentionné, 'lemmatized_tokens' contient les textes déjà nettoyés, tokenisés et lemmatisés
large_unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens'].values]
large_unlabeled_sequences = tokenizer.texts_to_sequences(large_unlabeled_texts)
large_unlabeled_X = pad_sequences(large_unlabeled_sequences, maxlen=small_X.shape[1])

# Utiliser le modèle pour prédire les étiquettes de la grande dataset
large_predictions = model.predict(large_unlabeled_X)

# Convertir les prédictions en étiquettes lisibles
large_predicted_labels = np.argmax(large_predictions, axis=1)
large_predicted_labels = label_encoder.inverse_transform(large_predicted_labels)

# Ajouter les étiquettes prédites à la grande dataset
df['predicted_score_lstm'] = large_predicted_labels


Epoch 1/10
81/81 [==============================] - 3s 12ms/step - loss: 1.7517 - accuracy: 0.3322
Epoch 2/10
81/81 [==============================] - 1s 12ms/step - loss: 1.2053 - accuracy: 0.5925
Epoch 3/10
81/81 [==============================] - 1s 12ms/step - loss: 0.8178 - accuracy: 0.7242
Epoch 4/10
81/81 [==============================] - 1s 12ms/step - loss: 0.5852 - accuracy: 0.8108
Epoch 5/10
81/81 [==============================] - 1s 12ms/step - loss: 0.4696 - accuracy: 0.8508
Epoch 6/10
81/81 [==============================] - 1s 12ms/step - loss: 0.3851 - accuracy: 0.8745
Epoch 7/10
81/81 [==============================] - 1s 12ms/step - loss: 0.3284 - accuracy: 0.8970
Epoch 8/10
81/81 [==============================] - 1s 12ms/step - loss: 0.2987 - accuracy: 0.9060
Epoch 9/10
81/81 [==============================] - 1s 12ms/step - loss: 0.2654 - accuracy: 0.9122
Epoch 10/10
2184/2184 [==============================] - 5s 2ms/step
Epoch 1/10
2265/2265 [==================

In [ ]:
predicted_labels

# Semi-supervised Learning with TfidfVectorizer and Naive Bayes

In [ ]:
# Préparation des données étiquetées
small_texts = df_labled['comment'].values
small_labels = df_labled['score'].values

# Création d'un modèle - pipeline TF-IDF suivi d'un Naive Bayes
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Entraînement du modèle avec les données étiquetées
model.fit(small_texts, small_labels)

# Prédire sur la grande dataset
unlabeled_texts = [' '.join(tokens) for tokens in df['lemmatized_tokens'].values]

# Utiliser le modèle pour prédire les étiquettes de la grande dataset
large_predicted_labels = model.predict(unlabeled_texts)

# Ajouter les étiquettes prédites à la grande dataset
df['predicted_score_TfidfVectorizer'] = large_predicted_labels


In [ ]:
df

# Semi-supervised Learning with XGBoost


In [ ]:
import numpy as np
import xgboost as xgb
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Créer des embeddings de mots avec Word2Vec
sentences = [text.split() for text in df_labled['comment'].values]
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec.model")

# Convertir les textes en représentations vectorielles en utilisant les embeddings de mots
def text_to_vector(text, model):
    words = text.split()
    vector = np.zeros(model.vector_size)
    for word in words:
        if word in model.wv:
            vector += model.wv[word]
    return vector / (len(words) + 1e-5)

X = np.array([text_to_vector(text, word2vec_model) for text in df_labled['comment'].values])
small_labels = df_labled['score'].values

# Encodez les étiquettes en valeurs numériques
label_encoder = preprocessing.LabelEncoder()
encoded_labels = label_encoder.fit_transform(small_labels)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# Entraîner un modèle XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': len(label_encoder.classes_),
    'max_depth': 6,
    'eta': 0.01,
    'subsample': 0.8
}
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
bst = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=10, evals=[(dtest, 'eval')])

# Évaluer les performances sur l'ensemble de test
y_pred = bst.predict(dtest)
print(classification_report(y_test, y_pred))

# Préparer les données non étiquetées
X_unlabeled = np.array([text_to_vector(' '.join(tokens), word2vec_model) for tokens in df['lemmatized_tokens'].values])

# Utiliser le modèle pour prédire les étiquettes de la grande dataset
d_unlabeled = xgb.DMatrix(X_unlabeled)
large_predictions = bst.predict(d_unlabeled)

# Convertir les prédictions en étiquettes lisibles
large_predicted_labels = label_encoder.inverse_transform(large_predictions.astype(int))

# Ajouter les étiquettes prédites à la grande dataset
df['predicted_score_XGB'] = large_predicted_labels


# Semi-supervised Learning with H2O AUTOML


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

# Initialise H2O
h2o.init()

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = vectorizer.fit_transform(df_labled['comment'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names())

# Ajouter les étiquettes de score à la DataFrame
tfidf_df['score'] = df_labled['score']

# Convertir en H2OFrame
data = h2o.H2OFrame(tfidf_df)

# Spécifiez les noms des colonnes pour les features et la cible
features = vectorizer.get_feature_names()
target = "score"

# Convertir la colonne cible en un facteur
data[target] = data[target].asfactor()

# Exécuter AutoML
aml = H2OAutoML(max_models=10, max_runtime_secs=300, seed=1)
aml.train(y=target, x=features, training_frame=data)

# Afficher le classement des modèles
lb = aml.leaderboard
print(lb)

# Récupérer le meilleur modèle par son ID
best_model = h2o.get_model(aml.leaderboard[0, 'model_id'])

# Convertir le grand ensemble de données (df) en H2OFrame
large_data = h2o.H2OFrame(df)

# Sélectionnez la colonne 'lemmatized_tokens' pour les prédictions
new_data = large_data[:, ["lemmatized_tokens"]]

# Faire des prédictions en utilisant le meilleur modèle
predictions = best_model.predict(new_data)

# Convertir les prédictions H2OFrame en un DataFrame pandas
predictions_df = predictions.as_data_frame()

# Créer un DataFrame de sortie avec 'lemmatized_tokens' et 'predicted_score_H2O'
output_df = pd.DataFrame({"lemmatized_comment": df["lemmatized_tokens"], "predicted_score_H2O": predictions_df["predict"]})

# Afficher le DataFrame de sortie
print(output_df)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,47 mins 36 secs
H2O_cluster_timezone:,Africa/Lagos
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,2 months
H2O_cluster_name:,H2O_from_python_msi_leu65e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.688 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


C:\Users\msi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


                                             comment        score
0   aussi absorption directement niveau peau dit ...  Réclamation
1                 doute plus pour efficacité produit  Réclamation
2                       jai réclamé régler situation        Refus
3                 jamais reçu échantillon auparavant        Refus
4      même sil observe antécédent chéloïde ème jour  Réclamation


In [ ]:
# Écrire le DataFrame avec les deux colonnes dans un fichier Excel
df[['comment', 'predicted_score_lstm','predicted_score_TfidfVectorizer','predicted_score_XGB']].to_excel('resources/dev_labo/data/processed/Comments_Clasification.xlsx', index=False)